# Trabajo Practico 1
## Maximización del Ratio de Sharpe con Derivadas Numéricas

El ratio de Sharpe está definido como:

$$
S(w) = \frac{w^\top \mu - r_f}{\sqrt{w^\top \Sigma w}}
$$

donde:

- **\( w \)** es el vector de pesos del portafolio,  
- **\( \mu \)** es el vector de rendimientos esperados,  
- **\( r_f \)** es la tasa libre de riesgo,  
- **\( \Sigma \)** es la matriz de covarianzas.

# Ejercicio 1

In [9]:
import numpy as np


class portfolio():
    def __init__(self, weights, returns, rf):
        # Weighst = tupla con los w de los 3 activos
        # Returns = Lista de 3 tuplas con los retornos de los 3 activos
        self.returns = np.array(returns)
        self.mus = self.mus
        if len(weights)==2:
            weights.np.append(1-self.weights.sum())
        self.weights = np.array(weights)
        self.rf = rf
        self.cov_matrix = np.cov(returns)

    def mus(self):
        prom_r = []
        for i in self.returns:
            i = np.mean(i)
            prom_r.append(i)
        return np.array(prom_r)

    def portfolio_return(self):
        return self.weights.T @ self.mus()

    def portfolio_volatility(self):
        return np.sqrt(self.weights.T @ self.cov_matrix @ self.weights)

    def sharpe_ratio(self):
        return (self.portfolio_return() - self.rf) / self.portfolio_volatility()
    
weights = (0.2, 0.5, 0.3)
returns = [(0.1, 0.2, 0.15, 0.12, -0.1), (0.05, 0.1, 0.12, 0.08, -0.05), (0.2, 0.25, 0.22, 0.21, 0.15)]
rf = 0.03
portfolio_prueba = portfolio(weights, returns, rf)

print("Sharpe Ratio:", portfolio_prueba.sharpe_ratio())
print("Portfolio Return:", portfolio_prueba.portfolio_return())


Sharpe Ratio: 1.2167094660038238
Portfolio Return: 0.11060000000000002


# Ejercicio 2

In [ ]:
#Punto 2 
def dsharpe(w):
    """
    Esta función hace la derivada centrada del
    ratio de Sharpe, evaluada en el punto que
    el usuario ingrese.
    """
    h = 1e-4  # El movimiento marginal de la derivada
    # Asumiendo que 'portfolio_prueba' es el objeto de portfolio
    return (portfolio_prueba.sharpe_ratio(w + h * np.array(w)) - portfolio_prueba.sharpe_ratio(w - h * np.array(w))) / (2 * h * np.array(w))

def gdscent(w0):
    """
    Esta función recibe por parte del usuario
    un vector de ponderadores inicial del 
    portafolio. Mediante el uso de derivadas 
    centradas y el algoritmo de descenso por
    gradiente de -f(X) maximiza el ratio de 
    Sharpe.

    w = (w1, w2) --> Trabajamos en R**2, y 
    seteamos w3 = 1-w1-w2, permitiendo el
    short selling y cumpliendo implicitamente
    con la restricción w1 + w2 + w3 = 1.

    Notación:
        Y = el valor de la función evaluada en un punto 
        X = el vector o punto 
    """
    tol = e-16 #Fijarse si es 6 o 16
    e =  e-4 #Calibrar segun convenga
    #Mi X0 vendría a ser w
    Y0 = - self.sharpe(w0) 
    w1 = w0 - e*self.dsharpe(w0)
    Y1 = - self.sharpe(w1)
    if abs(Y1-Y0) < tol:
        return w1
    else:
        self.gdscent(w1)